In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import pandas as pd

import time
import datetime

In [2]:
chrome_options = Options()
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--window-size=1920x1080")

___
# Get news links

In [3]:
locales = {
    'eco': ['https://vtv.vn/kinh-te.htm', 'cache/links_eco.csv', 'data/vtv_eco_raw.csv'],
    'social': ['https://vtv.vn/xa-hoi.htm', 'cache/links_social.csv', 'data/vtv_social_raw.csv']
}

In [4]:
def extract_summary(text_: str):
    text = text_[:]
    text = text.split(' - ')
    text.pop(0)
    text = ' '.join(text)
    return text

In [5]:
def get_link_until_date(url, date: datetime.date, sleep_interval):
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        driver.find_element(by=By.CLASS_NAME, value='loadmore').click()
        time.sleep(sleep_interval)
        upto_date = driver.find_elements(by=By.CLASS_NAME, value='tlitem')[-2].find_element(by=By.CLASS_NAME, value='time').text
        [dd, mm, yyyy] = upto_date.split('/')
        dd = int(dd)
        mm = int(mm)
        yyyy = int(yyyy)
        if date > datetime.date(yyyy, mm, dd): 
            elements = driver.find_elements(by=By.CLASS_NAME, value='tlitem')
            for element in elements[::-1]:
                try:
                    e_date = element.find_element(by=By.CLASS_NAME, value='time').text
                    [day, month, year] = e_date.split('/')
                    day, month, year = int(day), int(month), int(year)
                    if datetime.date(year, month, day) < date:
                        elements.remove(element)
                    else: 
                        break
                except: elements.remove(element)
            break
    
    links = []
    dates = []
    summaries = []
    for element in elements:
        try:
            link = element.find_elements(by=By.TAG_NAME, value='a')[0].get_attribute('href')
            date = element.find_element(by=By.CLASS_NAME, value='time').text
            summary = extract_summary(element.find_element(by=By.CLASS_NAME, value='sapo').text)
            links.append(link)
            dates.append(date)
            summaries.append(summary)
        except: pass
    
    return pd.DataFrame({'link': links, 'date': dates, 'summary': summaries})

___
# From link to data

In [6]:
def from_links_to_raw_data(df, loading_interval=3):
    links = df['link'].values
    dates = df['date'].values
    summaries = df['summary'].values
    
    driver = webdriver.Chrome(options=chrome_options)
    driver.execute_cdp_cmd('Page.addScriptToEvaluateOnNewDocument', {'source': 'setTimeout(function(){window.stop();}, 4000);'})
    titles = []
    contents = []
    for url in links:
        driver.get(url)
        time.sleep(loading_interval)
        
        driver.execute_script("""
            var element = document.querySelector(".VCSortableInPreviewMode");
            if (element)
                element.parentNode.removeChild(element);
            """)
        
        try:
            content = driver.find_element(by=By.CLASS_NAME, value='noidung')
            title = content.find_element(by=By.CSS_SELECTOR, value='h1.title_detail').text
            paragraphs = content.find_elements(by=By.CSS_SELECTOR, value='#entry-body p')
            try:
                text = [content.find_element(by=By.CSS_SELECTOR, value='h2.sapo').text.split(' - ')[1]]
            except: text = []
            for paragraph in paragraphs:
                text.append(paragraph.text)
            titles.append(title)
            contents.append(" ".join(text))
        except:
            contents.append(None)
            titles.append(None)
    
    return pd.DataFrame({'title': titles, 'content': contents, 'summary': summaries, 'date': dates, 'url': links})

In [7]:
def add_to_data(df: pd.DataFrame, path, reset=False):
    if reset == True: df.to_csv(path, index=False)
    else: df.to_csv(path, mode='a', header=False, index=False)

In [13]:
def crawl(categorical, date):
    [url, link_path, raw_data_path] = locales[categorical]
    current_url = pd.read_csv(link_path)
    url_df = get_link_until_date(url, date, 3)
    url_df = url_df.merge(current_url, on=['link', 'date', 'summary'], how='left', indicator=True)
    url_df = url_df[url_df['_merge'] == 'left_only'].drop('_merge', axis=1)
    if len(url_df) == 0: return
    df = from_links_to_raw_data(url_df)
    url_df.to_csv(link_path, index=False, header=False, mode='a')
    add_to_data(df, reset=False, path=raw_data_path)
    
def crawl_from_link(categorical):
    [_, link_path, raw_data_path] = locales[categorical]
    url_df = pd.read_csv(link_path)
    df = from_links_to_raw_data(url_df)
    add_to_data(df, reset=False, path=raw_data_path)

In [15]:
crawl('social', datetime.date(2024, 6, 2))

In [12]:
crawl_from_link('social')
crawl_from_link('eco')

# Merge Data

In [1]:
import pandas as pd

In [16]:
eco_data = pd.read_csv(locales['eco'][2])
social_data = pd.read_csv(locales['social'][2])

In [17]:
eco_data = eco_data.assign(categorical='economic')
social_data = social_data.assign(categorical='social')

In [18]:
eco_data

,title,content,summary,date,url,categorical
0,Ưu đãi khách hàng vay vốn phục vụ nhu cầu đời ...,Agribank dành khoảng 10.000 tỷ đồng triển khai...,Agribank dành khoảng 10.000 tỷ đồng triển khai...,04/04/2024,https://vtv.vn/kinh-te/uu-dai-khach-hang-vay-v...,economic
1,Thu hút FDI tăng cả lượng lẫn chất,"Quý I năm 2024, đã có hơn 6,17 tỷ USD vốn đầu ...","Quý I năm 2024, đã có hơn 6,17 tỷ USD vốn đầu ...",04/04/2024,https://vtv.vn/kinh-te/thu-hut-fdi-tang-ca-luo...,economic
2,Bộ Tài chính đề xuất bãi bỏ 5 Quyết định về gi...,Bộ Tài chính đề xuất bãi bỏ 6 Quyết định của T...,Bộ Tài chính đề xuất bãi bỏ 6 Quyết định của T...,04/04/2024,https://vtv.vn/kinh-te/bo-tai-chinh-de-xuat-ba...,economic
3,Forbes: Tất cả các tỷ phú dưới 30 tuổi trên th...,Một nghiên cứu của tạp chí kinh doanh Forbes m...,Một nghiên cứu của tạp chí kinh doanh Forbes m...,04/04/2024,https://vtv.vn/kinh-te/forbes-tat-ca-cac-ty-ph...,economic
4,Bất động sản kho lạnh hút vốn đầu tư,Kho lạnh Theo khảo sát từ một số công ty nghiê...,Kho lạnh sản phẩm dùng để trữ hàng hoá đang th...,04/04/2024,https://vtv.vn/kinh-te/bat-dong-san-kho-lanh-h...,economic
...,...,...,...,...,...,...
2186,Cơ hội về hưu sớm cho người thừa năm đóng BHXH,Tổng LĐLĐ Việt Nam đề xuất người lao động thừa...,Tổng LĐLĐ Việt Nam đề xuất người lao động thừa...,21/05/2024,https://vtv.vn/kinh-te/co-hoi-ve-huu-som-cho-n...,economic
2187,Hàn Quốc kỳ vọng xuất khẩu mì ăn liền vượt 1 t...,Cục Hải quan Hàn Quốc (KCS) dự kiến xuất khẩu ...,Cục Hải quan Hàn Quốc (KCS) dự kiến xuất khẩu ...,21/05/2024,https://vtv.vn/kinh-te/han-quoc-ky-vong-xuat-k...,economic
2188,Khoanh 28.380 tỷ đồng tiền nợ thuế sau 3 năm x...,Sau 3 năm thực hiện Nghị quyết số 94/2019/QH14...,Sau 3 năm thực hiện Nghị quyết số 94/2019/QH14...,21/05/2024,https://vtv.vn/kinh-te/khoanh-28380-ty-dong-ti...,economic
2189,"Hướng tín dụng vào sản xuất, phấn đấu giảm lãi...","Theo thống kê của Ngân hàng Nhà nước, đến giữa...","Theo thống kê của Ngân hàng Nhà nước, đến giữa...",21/05/2024,https://vtv.vn/kinh-te/huong-tin-dung-vao-san-...,economic


In [19]:
merge_data = pd.concat([eco_data, social_data], axis=0)

In [20]:
merge_data.dropna().drop_duplicates()

,title,content,summary,date,url,categorical
0,Ưu đãi khách hàng vay vốn phục vụ nhu cầu đời ...,Agribank dành khoảng 10.000 tỷ đồng triển khai...,Agribank dành khoảng 10.000 tỷ đồng triển khai...,04/04/2024,https://vtv.vn/kinh-te/uu-dai-khach-hang-vay-v...,economic
1,Thu hút FDI tăng cả lượng lẫn chất,"Quý I năm 2024, đã có hơn 6,17 tỷ USD vốn đầu ...","Quý I năm 2024, đã có hơn 6,17 tỷ USD vốn đầu ...",04/04/2024,https://vtv.vn/kinh-te/thu-hut-fdi-tang-ca-luo...,economic
2,Bộ Tài chính đề xuất bãi bỏ 5 Quyết định về gi...,Bộ Tài chính đề xuất bãi bỏ 6 Quyết định của T...,Bộ Tài chính đề xuất bãi bỏ 6 Quyết định của T...,04/04/2024,https://vtv.vn/kinh-te/bo-tai-chinh-de-xuat-ba...,economic
3,Forbes: Tất cả các tỷ phú dưới 30 tuổi trên th...,Một nghiên cứu của tạp chí kinh doanh Forbes m...,Một nghiên cứu của tạp chí kinh doanh Forbes m...,04/04/2024,https://vtv.vn/kinh-te/forbes-tat-ca-cac-ty-ph...,economic
4,Bất động sản kho lạnh hút vốn đầu tư,Kho lạnh Theo khảo sát từ một số công ty nghiê...,Kho lạnh sản phẩm dùng để trữ hàng hoá đang th...,04/04/2024,https://vtv.vn/kinh-te/bat-dong-san-kho-lanh-h...,economic
...,...,...,...,...,...,...
2562,Hà Nội miễn phí cấp phiếu lý lịch tư pháp trên...,"Theo đó, từ ngày 1/6 đến hết ngày 31/12/2024, ...","Theo đó, từ ngày 1/6 đến hết ngày 31/12/2024, ...",02/06/2024,https://vtv.vn/xa-hoi/ha-noi-mien-phi-cap-phie...,social
2563,"Thời tiết ngày 2/6: Hà Nội ngày nắng, chiều tố...","Thời tiết ngày 6/2, nhiều khu vực ngày nắng, c...","Thời tiết ngày 6/2, nhiều khu vực ngày nắng, c...",02/06/2024,https://vtv.vn/xa-hoi/thoi-tiet-ngay-2-6-ha-no...,social
2564,Rủi ro từ các dịch vụ thẩm mỹ quảng cáo sai sự...,"Nắm được vào tâm lý khách hàng, nhiều cơ sở th...","Nắm được vào tâm lý khách hàng, nhiều cơ sở th...",02/06/2024,https://vtv.vn/xa-hoi/rui-ro-tu-cac-dich-vu-th...,social
2565,"Nhiều điểm mới trong dự án Luật Quản lý, sử dụ...",Các cơ quan chức năng đề xuất dao có tính sát ...,Các cơ quan chức năng đề xuất dao có tính sát ...,02/06/2024,https://vtv.vn/xa-hoi/nhieu-diem-moi-trong-du-...,social


In [21]:
merge_data.rename(columns={
    'title': 'Title',
    'content': 'Content',
    'summary': 'Summary',
    'date': 'Date',
    'url': 'Url',
    'categorical': 'Categorical'
}, inplace=True)

In [22]:
merge_data.dropna(inplace=True)
merge_data.drop_duplicates(inplace=True)

In [25]:
merge_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3469 entries, 0 to 2566
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        3469 non-null   object
 1   Content      3469 non-null   object
 2   Summary      3469 non-null   object
 3   Date         3469 non-null   object
 4   Url          3469 non-null   object
 5   Categorical  3469 non-null   object
dtypes: object(6)
memory usage: 189.7+ KB


In [26]:
merge_data.to_csv('data/news.csv', index=False)